In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker


In [12]:
Base = declarative_base()

class Student(Base):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    last_name = Column(String)
    first_name = Column(String)
    faculty = Column(String)
    course = Column(String)
    grade = Column(Integer)

    def __repr__(self):
        return f"{self.last_name} {self.first_name}, {self.faculty}, {self.course}, {self.grade}"


In [13]:
class StudentManager:
    def __init__(self, db_url='sqlite:///students.db'):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def insert_students(self, students_list):
        session = self.Session()
        for data in students_list:
            student = Student(**data)
            session.add(student)
        session.commit()
        session.close()

    def get_by_faculty(self, faculty):
        session = self.Session()
        result = session.query(Student).filter_by(faculty=faculty).all()
        session.close()
        return result

    def get_unique_courses(self):
        session = self.Session()
        result = set([row.course for row in session.query(Student.course).distinct()])
        session.close()
        return result

    def get_avg_grade_by_faculty(self, faculty):
        session = self.Session()
        result = session.query(Student).filter_by(faculty=faculty).all()
        session.close()
        grades = [s.grade for s in result]
        return sum(grades) / len(grades) if grades else None

    def get_lowscore_by_course(self, course, threshold=30):
        session = self.Session()
        result = session.query(Student).filter_by(course=course).filter(Student.grade < threshold).all()
        session.close()
        return result


In [14]:
import shutil
shutil.copy("C:/Users/User/Downloads/students.csv", "./students.csv")

'./students.csv'

In [15]:
df = pd.read_csv('students.csv')
df.columns = ['last_name','first_name','faculty','course','grade']

students_list = df.to_dict(orient="records")
manager = StudentManager()
manager.insert_students(students_list)


In [16]:
print("Студенты факультета ФПМИ:")
for student in manager.get_by_faculty("ФПМИ"):
    print(student)

print("Уникальные курсы:", manager.get_unique_courses())

print("Средний балл на ФЛА:", manager.get_avg_grade_by_faculty("ФЛА"))

print("Студенты курса 'Теор. Механика' с оценкой < 30:")
for student in manager.get_lowscore_by_course("Теор. Механика"):
    print(student)


Студенты факультета ФПМИ:
Ким Петр, ФПМИ, Мат. Анализ, 28
Смит Федор, ФПМИ, Мат. Анализ, 51
Смит Федор, ФПМИ, Физика, 95
Смит Федор, ФПМИ, Информатика, 3
Смит Федор, ФПМИ, История, 93
Райт Вероника, ФПМИ, Теор. Механика, 0
Райт Вероника, ФПМИ, Мат. Анализ, 47
Райт Вероника, ФПМИ, История, 11
Райт Вероника, ФПМИ, Психология, 68
Джонс Андрей, ФПМИ, Физика, 31
Джонс Андрей, ФПМИ, Информатика, 46
Джонс Андрей, ФПМИ, Психология, 85
Чан Алексей, ФПМИ, История, 32
Чан Алексей, ФПМИ, Физика, 8
Чан Алексей, ФПМИ, Информатика, 42
Чжоу Мария, ФПМИ, Теор. Механика, 15
Чжоу Мария, ФПМИ, Мат. Анализ, 13
Чжоу Мария, ФПМИ, Психология, 75
Браун Николай, ФПМИ, Физика, 85
Браун Николай, ФПМИ, Теор. Механика, 48
Браун Николай, ФПМИ, Информатика, 51
Тан Екатеринар, ФПМИ, Мат. Анализ, 7
Тан Екатеринар, ФПМИ, Психология, 52
Райт Вероника, ФПМИ, Психология, 89
Чан Алексей, ФПМИ, Информатика, 35
Райт Вероника, ФПМИ, Физика, 75
Смит Федор, ФПМИ, Психология, 91
Смит Федор, ФПМИ, Мат. Анализ, 57
Смит Федор, ФПМИ,